In [0]:
%fs ls /Workspace/Users/kanikr1810@gmail.com/

In [0]:
%pip install torch torchvision transformers opencv-python streamlit pillow scikit-learn

In [0]:
dbutils.library.restartPython()

In [0]:
%pip install transformers

In [0]:
%pip show torch torchvision streamlit

In [0]:

%pip install kaggle

In [0]:
dbutils.library.restartPython()

In [0]:


!mkdir -p /tmp/.kaggle
with open('/tmp/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"kanimozhikr","key":"b55c134d870dc6150c8f21e49f30fb35"}')




In [0]:

!kaggle datasets download -d anujms/car-damage-detection -p /tmp/
!unzip /tmp/car-damage-detection.zip -d /Workspace/Users/kanikr1810@gmail.com/


In [0]:
!ls /Workspace/Users/kanikr1810@gmail.com/data1a/training/00-damage/

In [0]:

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

# Path to dataset
data_dir = "/Workspace/Users/kanikr1810@gmail.com/data1a/"

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import transforms

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pretrained ResNet50 for fraud detection
fraud_model = resnet50(weights=ResNet50_Weights.DEFAULT)
for param in fraud_model.parameters():
    param.requires_grad = False
fraud_model.fc = nn.Linear(fraud_model.fc.in_features, 3)
fraud_model = fraud_model.to(device)

# Define loss and optimizer for fraud model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fraud_model.fc.parameters(), lr=0.001)

# Training loop for fraud detection
epochs = 5
for epoch in range(epochs):
    fraud_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = fraud_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Validation for fraud detection
fraud_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = fraud_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy (Fraud): {100 * correct / total:.2f}%")

# Save the trained fraud model
torch.save(fraud_model.state_dict(), "/Workspace/Users/kanikr1810@gmail.com/fraud_detector.pth")
print("Fraud model saved at /Workspace/Users/kanikr1810@gmail.com/")
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy (Fraud): {100 * correct / total:.2f}%")

# Example: Car detection on validation set
car_model.eval()
car_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
car_correct = 0
car_total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        # Assuming car labels: 1 for car, 0 for not car (update as per your dataset)
        outputs = car_model(images)
        _, predicted = torch.max(outputs, 1)
        car_total += labels.size(0)
        # If your dataset has car/not car labels, update this comparison
        car_correct += (predicted == labels).sum().item()

print(f"Validation Accuracy (Car Detection): {100 * car_correct / car_total:.2f}%")

# Save the trained fraud model
torch.save(fraud_model.state_dict(), "/Workspace/Users/kanikr1810@gmail.com/fraud_detector.pth")
print("Fraud model saved at /Workspace/Users/kanikr1810@gmail.com/")

# Save the car detection model (if fine-tuned)
torch.save(car_model.state_dict(), "/Workspace/Users/kanikr1810@gmail.com/car_detector.pth")
print("Car detector model saved at /Workspace/Users/kanikr1810@gmail.com/")

In [0]:

%sh
cp /Workspace/Users/kanikr1810@gmail.com/fraud_detector.pth ./fraud_detector.pth

%sh
cp /Workspace/Users/kanikr1810@gmail.com/car_detector.pth ./car_detector.pth



In [0]:
%pip install streamlit pyngrok torch torchvision pillow

In [0]:
print("Hi")

In [0]:
dbutils.library.restartPython()

In [0]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torchvision import transforms
from PIL import Image

# Load model architecture as in cell 12
class FraudDetector(nn.Module):
    def __init__(self):
        super(FraudDetector, self).__init__()
        self.model = resnet50()
        self.model.fc = nn.Linear(self.model.fc.in_features, 3)  # 3 classes

    def forward(self, x):
        return self.model(x)

# Load trained weights
model_path = "./fraud_detector.pth"
model = FraudDetector()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# Streamlit UI
st.title("Car Insurance Fraud Detection")
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        # Map indices to class names (update as per your dataset)
        class_names = ["Genuine", "Minor Fraud", "Major Fraud"]
        label = class_names[pred_idx]
        st.success(f"Prediction: {label}")

In [0]:

%sh
cp /tmp/fraud_detector.pth ./fraud_detector.pth


In [0]:


import base64

file_path = "/tmp/fraud_detector.pth"  # Change if needed
with open(file_path, "rb") as f:
    data = f.read()

b64 = base64.b64encode(data).decode()
download_link = f"data:application/octet-stream;base64,{b64}Click here to download fraud_detector.pth</a>"
displayHTML(download_link)


In [0]:

import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Paths
data_dir = "/Workspace/Users/kanikr1810@gmail.com/data1a/"
save_dir = "/Workspace/Users/kanikr1810@gmail.com/models"
os.makedirs(save_dir, exist_ok=True)

# Reproducibility
seed = 42
torch.manual_seed(seed)

# Transforms
img_size = 224
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

eval_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Datasets (use same folder structure: root/class_name/*.jpg)
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

# Split sizes
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

# Split with fixed seed
generator = torch.Generator().manual_seed(seed)
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size], generator=generator)

# For val/test, override transform to deterministic
val_dataset.dataset.transform = eval_transform
test_dataset.dataset.transform = eval_transform

# Dataloaders
batch_size = 32
num_workers = 4  # tweak per cluster

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

class_names = full_dataset.classes
num_classes = len(class_names)
print(f"Classes ({num_classes}): {class_names}")


In [0]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

# Paths
data_dir = "/Workspace/Users/kanikr1810@gmail.com/data1a/"
save_dir = "/Workspace/Users/kanikr1810@gmail.com/models"
os.makedirs(save_dir, exist_ok=True)

# Reproducibility
seed = 42
torch.manual_seed(seed)

# Transforms
img_size = 224
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

eval_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Datasets (folder structure: root/class_name/*.jpg)
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
class_names = full_dataset.classes
num_classes = len(class_names)
print(f"Classes ({num_classes}): {class_names}")

# Split sizes
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

generator = torch.Generator().manual_seed(seed)
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size], generator=generator)
val_dataset.dataset.transform = eval_transform
test_dataset.dataset.transform = eval_transform

batch_size = 32
num_workers = 0  # Use single-process data loading

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

# Model: ResNet50 for 3 classes ["not_car", "ai_generated_car", "ai_edited_car"]
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total
    val_loss, val_correct, val_total = 0.0, 0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total
    print(f"Epoch {epoch+1}: Train Acc {train_acc:.3f}, Val Acc {val_acc:.3f}")

# Save model
torch.save(model.state_dict(), os.path.join(save_dir, "car_type_detector_resnet50.pth"))
print(f"Model saved at {save_dir}/car_type_detector_resnet50.pth")

In [0]:

%pip install transformers streamlit


In [0]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Class names
class_names = ["not_car", "ai_generated_car", "ai_edited_car"]

# Model definition: ResNet50
class CarTypeDetector(nn.Module):
    def __init__(self, num_classes=3):
        super(CarTypeDetector, self).__init__()
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    def forward(self, x):
        return self.model(x)

# Load ResNet50 model
resnet_model_path = "./car_type_detector_resnet50.pth"
resnet_model = CarTypeDetector(num_classes=len(class_names))
resnet_model.load_state_dict(torch.load(resnet_model_path, map_location=torch.device('cpu')))
resnet_model.eval()

# Load OpenAI CLIP model
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# Image transform for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Streamlit UI
st.title("Car Type Detection: Not Car / AI Generated / AI Edited")
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # ResNet50 prediction
    img_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = resnet_model(img_tensor)
        probs = torch.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        label = class_names[pred_idx]
        st.success(f"ResNet50 Prediction: {label}")
        st.write(f"ResNet50 Probabilities: {dict(zip(class_names, probs.squeeze().tolist()))}")

    # CLIP prediction
    clip_inputs = clip_processor(
        text=class_names,
        images=image,
        return_tensors="pt",
        padding=True
    )
    with torch.no_grad():
        clip_outputs = clip_model(**clip_inputs)
        logits_per_image = clip_outputs.logits_per_image
        clip_probs = logits_per_image.softmax(dim=1).squeeze().tolist()
        clip_pred_idx = logits_per_image.argmax(dim=1).item()
        clip_label = class_names[clip_pred_idx]
        st.success(f"CLIP Prediction: {clip_label}")
        st.write(f"CLIP Probabilities: {dict(zip(class_names, clip_probs))}")